In [ ]:
!pip install torch transformers accelerate bitsandbytes unsloth

In [ ]:
# Cargar modelo y tokenizador
from transformers import AutoModel, AutoTokenizer
from unsloth import FastLanguageModel
import torch

model_name = "AdrianML7/asesor-finanzas-phi3"
#HF_TOKEN = "TOKEN"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, #Repo publico
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model)


/tmp/ipython-input-602156311.py:3: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Mistral patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32009)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj

In [3]:
#Nuestro dataset tiene instruccion, input y output
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. The output must have a clean structure
separated by paragraphs.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

instruction = input("Cual es tu pregunta? ")
input_context = input("Cual es tu perfil? ")

Cual es tu pregunta?Como ahorrar para la entrada de una vivienda?
Cual es tu perfil?Joven estudiante en prácticas, 800 euros de ahorro mensuales


In [4]:
#Preparar prompt

#Adapta el prompt y tokeniza
prompt = alpaca_prompt.format(instruction, input_context, "")
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

In [ ]:
#Prepara la respuesta y muestrala por pantalla
from IPython.display import Markdown, display

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
texto_crudo = tokenizer.batch_decode(outputs)[0]

# Limpiamos para quedarnos solo con la respuesta del asistente
respuesta_limpia = texto_crudo.split("### Response:\n")[-1].replace("<|endoftext|>", "")

# Renderizar Markdown
display(Markdown(respuesta_limpia))

Entiendo que, como estudiante, buscas construir un colchón para tu futuro inmobiliario. Lo primero es definir un objetivo claro: por ejemplo, si deseas comprar en 5 años, necesitarás acumular alrededor de 40 000 € (asumiendo un precio de 80 000 € y un enganche del 20 %). Con 800 € al mes dispones de 9 600 € al año, lo que equivale a unos 720 € mensuales después de impuestos (suponiendo una tasa del 20 %). 

**1. Fondo de emergencia**
- Antes de invertir, destina entre 3 y 6 meses de gastos a una cuenta de alta liquidez (cuenta de ahorros o depósito a corto plazo). Esto te protege ante imprevistos y evita que tengas que vender inversiones en momentos desfavorables.

**2. Ahorro sistemático**
- Programa una domiciliación automática de 720 € al mes a una cuenta de ahorro o a un plan de pensiones con ventajas fiscales. La automatización reduce la tentación de gastar ese dinero.

**3. Inversión de bajo riesgo**
- Con un horizonte de 5 años y un perfil conservador, puedes destinar una parte (por ejemplo, 50 % del ahorro mensual) a instrumentos de renta fija de corto plazo, como bonos del Estado o fondos de bonos de alta calidad. Estos ofrecen rendimientos modestos pero estables y bajos volatilidad.
- El resto (50 %) puede ir a un fondo indexado de acciones de gran capitalización (ej. MSCI World) con exposición global, que históricamente ha superado la inflación a medio‑largo plazo. Aunque la volatilidad es mayor, el horizonte de 5 años permite suavizar los altibajos.

**4. Revisión y ajuste**
- Cada 12 meses revisa tu progreso: si tu ahorro supera el objetivo, puedes aumentar el enganche o destinar más a inversiones de mayor potencial de crecimiento.
- Si en algún momento tu situación cambia (p. ej., ingresos mayores o gastos inesperados), ajusta el porcentaje destinado a ahorro y a inversión.

**Advertencia de riesgo**
- Todas las inversiones conllevan riesgo de pérdida de capital. Los fondos de renta fija pueden verse afectados por cambios en las tasas de interés, y los fondos indexados de acciones pueden experimentar caídas de valor en el corto plazo. Asegúrate de que el dinero que no necesites en los próximos 5 años no esté en instrumentos de alta volatilidad.

Con disciplina y una estrategia bien estructurada, podrás acumular el enganche necesario para tu vivienda sin comprometer tu estabilidad financiera actual.